In [67]:
import pandas as pd
import plotly.express as px

In [68]:
# Организуем чтение CSV из Google Drive по ссылке
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [69]:
# Т.к. файл train больше 5 гб, для его чтения выбрем первые 1000000 строк
train = pd.read_csv('/content/drive/MyDrive/train-001.csv', nrows=1000000)
lectures = pd.read_csv('/content/lectures.csv')
questions = pd.read_csv('/content/questions.csv')

In [70]:
# Соберем общую информацию
train.info()
lectures.info()
questions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 10 columns):
 #   Column                          Non-Null Count    Dtype  
---  ------                          --------------    -----  
 0   row_id                          1000000 non-null  int64  
 1   timestamp                       1000000 non-null  int64  
 2   user_id                         1000000 non-null  int64  
 3   content_id                      1000000 non-null  int64  
 4   content_type_id                 1000000 non-null  int64  
 5   task_container_id               1000000 non-null  int64  
 6   user_answer                     1000000 non-null  int64  
 7   answered_correctly              1000000 non-null  int64  
 8   prior_question_elapsed_time     976277 non-null   float64
 9   prior_question_had_explanation  996184 non-null   object 
dtypes: float64(1), int64(8), object(1)
memory usage: 76.3+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 4

In [71]:
# Посмотрим на количество пропусков в файле train
print(train.isnull().sum())

row_id                                0
timestamp                             0
user_id                               0
content_id                            0
content_type_id                       0
task_container_id                     0
user_answer                           0
answered_correctly                    0
prior_question_elapsed_time       23723
prior_question_had_explanation     3816
dtype: int64


In [72]:
# Посмотрим на количество данных
print(lectures.shape)
print(questions.shape)
print(train.shape)

(418, 4)
(13523, 5)
(1000000, 10)


In [73]:
# Посмотрим данные по лекциям
lectures.head()

,lecture_id,tag,part,type_of
0,89,159,5,concept
1,100,70,1,concept
2,185,45,6,concept
3,192,79,5,solving question
4,317,156,5,solving question


In [74]:
# Посмотрим данные по вопросам и ответам
questions.head()

,question_id,bundle_id,correct_answer,part,tags
0,0,0,0,1,51 131 162 38
1,1,1,1,1,131 36 81
2,2,2,0,1,131 101 162 92
3,3,3,0,1,131 149 162 29
4,4,4,3,1,131 5 162 38


In [75]:
# Посмотрим данные по работе студентов
train.head()

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,0,115,5692,0,1,3,1,NaN,NaN
1,1,56943,115,5716,0,2,2,1,37000.0,False
2,2,118363,115,128,0,0,0,1,55000.0,False
3,3,131167,115,7860,0,3,0,1,19000.0,False
4,4,137965,115,7922,0,4,1,1,11000.0,False


In [76]:
# Проанализируем % правильных ответов
correct_answers_rate = train['answered_correctly'].mean()
print(f'Процент правильных ответов: {correct_answers_rate * 100:.2f}%')

Процент правильных ответов: 61.75%


In [77]:
# Посмотрим среднее время, затраченное на ответ
time_answers_rate = train['prior_question_elapsed_time'].mean()
print(f'Среднее время на ответ: {time_answers_rate:.2f}')

Среднее время на ответ: 25302.96


In [78]:
# Проанализируем влияние наличия объяснений на правильность ответа
explanation_correct = train.groupby('prior_question_elapsed_time')['answered_correctly'].mean()
print(explanation_correct)

prior_question_elapsed_time
0.0         0.503147
200.0       1.000000
250.0       0.833333
333.0       0.647696
400.0       0.600000
              ...   
295000.0    0.750000
296000.0    0.857143
297000.0    1.000000
298000.0    0.500000
300000.0    0.666174
Name: answered_correctly, Length: 1659, dtype: float64


In [79]:
# Объединим данные для анализа вопросов
merged_df = pd.merge(train[train['content_type_id'] == 0], questions, left_on='content_id', right_on='question_id')
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 980093 entries, 0 to 980092
Data columns (total 15 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   row_id                          980093 non-null  int64  
 1   timestamp                       980093 non-null  int64  
 2   user_id                         980093 non-null  int64  
 3   content_id                      980093 non-null  int64  
 4   content_type_id                 980093 non-null  int64  
 5   task_container_id               980093 non-null  int64  
 6   user_answer                     980093 non-null  int64  
 7   answered_correctly              980093 non-null  int64  
 8   prior_question_elapsed_time     976277 non-null  float64
 9   prior_question_had_explanation  976277 non-null  object 
 10  question_id                     980093 non-null  int64  
 11  bundle_id                       980093 non-null  int64  
 12  correct_answer  

In [80]:
merged_df.head(100)

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,question_id,bundle_id,correct_answer,part,tags
0,0,0,115,5692,0,1,3,1,NaN,NaN,5692,5692,3,5,151
1,1,56943,115,5716,0,2,2,1,37000.0,False,5716,5716,2,5,168
2,2,118363,115,128,0,0,0,1,55000.0,False,128,128,0,1,131 149 92
3,3,131167,115,7860,0,3,0,1,19000.0,False,7860,7860,0,1,131 104 81
4,4,137965,115,7922,0,4,1,1,11000.0,False,7922,7922,1,1,131 149 92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,0,5382,5000,0,0,0,1,NaN,NaN,5000,5000,0,5,26
96,97,39828,5382,3944,0,1,1,0,24000.0,False,3944,3944,0,5,47
97,98,132189,5382,217,0,2,0,1,35000.0,False,217,217,0,2,143 71 29 102
98,99,153727,5382,5844,0,3,1,0,88000.0,False,5844,5844,0,5,91


In [81]:
# Анализ по тегам
tag_correctness = merged_df.groupby('tags')['prior_question_elapsed_time'].mean()
print(tag_correctness)

tags
1                       26494.408355
1 162                   32801.377038
10 111 92               24415.254237
10 164 102              21595.644970
10 164 162 29           21869.441667
                            ...     
98 97 21                41941.949721
98 97 35 117 122 162    42752.538462
98 97 35 146 122        45195.364706
98 97 46 135            50319.752475
98 97 77 135 162        46343.037736
Name: prior_question_elapsed_time, Length: 1501, dtype: float64


In [82]:
# Анализ по категориям лекций
part_correctness = merged_df.groupby('part')['prior_question_elapsed_time'].mean()
print(part_correctness)

part
1    22588.283868
2    18587.403019
3    23722.096672
4    23743.839112
5    25090.470269
6    32360.086186
7    45958.401765
Name: prior_question_elapsed_time, dtype: float64


In [83]:
# Анализ затраченного времени на количество правильных ответов
tag_correctness = merged_df.groupby('correct_answer')['answered_correctly'].mean()
print(tag_correctness)

correct_answer
0    0.660042
1    0.666288
2    0.610613
3    0.653545
Name: answered_correctly, dtype: float64


In [84]:
# Посчитаем % правильных ответов в объединенной таблице
all_question = merged_df['answered_correctly'].shape[0]
true_question  = merged_df[merged_df['answered_correctly'] == 1].shape[0]
print(f'Процент правильных ответов {(true_question *100)/all_question:.2f} %')

Процент правильных ответов 65.04 %


In [85]:
# Анализ по вопросам из одного пакета
tag_correctness = merged_df.groupby('task_container_id')['answered_correctly'].mean()
print(tag_correctness)

task_container_id
0       0.680530
1       0.515727
2       0.421643
3       0.543133
4       0.477714
          ...   
7735    1.000000
7736    1.000000
7737    1.000000
7738    1.000000
7739    1.000000
Name: answered_correctly, Length: 7740, dtype: float64


In [86]:
# Проанализируем влияние наличия объяснения на правильность ответа
explanation_correct =  merged_df.groupby('prior_question_had_explanation')['answered_correctly'].mean()
print(explanation_correct)

prior_question_had_explanation
False    0.488795
True     0.665913
Name: answered_correctly, dtype: float64


In [87]:
# Посмотрим, как зависит среднее количество правильных ответов от времени обучения
# Группируем данные по пользователям
result = (train.groupby('user_id')
    .agg(
        # количество строк
        rows=('row_id', 'count'),
        # максимальное значение времени
        time=('timestamp', 'max'),
        # среднее количество правильных ответов
        answered_correctly_mean=('answered_correctly', 'mean'),
    )
    .reset_index()
)

# Уберем из выборки результаты с малым количеством строк
result = result[result['rows'] > result['rows'].quantile(0.1)]

# Сглаживаем время с помощью целочисленного деления
result['time'] = (result['time'] // 1000000000)

# Группируем по времени и рассчитываем среднее количество правильных ответов
final_result = (result.groupby('time')
    .agg(
        answered_correctly=('answered_correctly_mean', 'mean'),
    )
    .reset_index()
    .sort_values(by='time')
)

# Строим график
fig = px.line(final_result, x='time', y='answered_correctly')
fig.show()

In [88]:
# Посмотрим на зависимость оценки от количества посещенных лекций
# Группируем данные по пользователям
result = (train.groupby('user_id')
    .agg(
        # количество строк
        rows=('row_id', 'count'),
        # количество лекций
        lectures=('content_type_id', lambda x: (x == 1).sum()),
        # количество вопросов
        questions=('content_type_id', lambda x: (x == 0).sum()),
        # количество правильных ответов
        correct=('answered_correctly', 'sum'),
    )
    .reset_index()
)

# Уберем из выборки результаты с малым количеством строк
result = result[(result['rows'] > result['rows'].quantile(0.1)) &
                (result['lectures'] < result['lectures'].quantile(0.95))]

# Считаем процент правильных ответов
result['correct_%'] = (result['correct'] / result['questions']) * 100

# Отбираем колонки для дальнейшего анализа и группируем по количеству лекций
final_result = (result.groupby('lectures')
    .agg(correct_mean=('correct_%', 'mean'))
    .reset_index()
    .sort_values('lectures')
)

# Построим график
import plotly.express as px

fig = px.line(final_result, x='lectures', y='correct_mean',
              labels={'correct_mean': 'Средний процент правильных ответов'})
fig.show()

# **Основные выводы для студента:**

1. Предшествующее объяснение влияет на правильность ответа.
2. Количество правильных ответов зависит от времени, затраченного на обучение.
3. Полученная оценка зависит от количества посещенных лекций.